In [ ]:
#pip install psycopg2-binary

In [82]:
import sqlalchemy
import pandas as pd
import psycopg2
import psycopg2.extras as extras 
from sqlalchemy import create_engine, inspect
from datetime import datetime

In [84]:
#Get DB credentials
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

In [33]:
#Insert dataframe to RDS using SQLAlchemy (We need to use psycopg2 for AWS Lambda instead of SQLAlchemy)
#conf ={
#    'host':DB_HOST,
#    'port':DB_PORT,
#    'database':DB_NAME,
#    'user':DB_USER,
#    'password':DB_PASSWORD
#}
#engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

#df.to_sql('bronze_airline_quality_reviews', engine, index=False, if_exists='replace')


#Insert dataframe to RDS using psycopg2 (We need to use psycopg2 for AWS Lambda instead of SQLAlchemy)
def insert_df_to_table(conn, df, table, needed_columns):
    tuples = [tuple(x) for x in df.to_numpy()] 

    # Get column names from DataFrame
    create_cols = '" TEXT,"'.join(needed_columns)
    cols = '","'.join(needed_columns)
    #print(cols)

    # Create table if it does not exist
    create_query = f'DROP TABLE IF EXISTS {table}; CREATE TABLE IF NOT EXISTS {table} ("{create_cols}" TEXT);'
    print(create_query)
    create_cursor = conn.cursor()

    
    # Insert values to table
    query = f'INSERT INTO {table} ("{cols}") VALUES %s'
    print(query)
    cursor = conn.cursor()
    
    try: 
        create_cursor.execute(create_query)
        
        # Execute the SQL query with the data
        extras.execute_values(cursor, query, tuples)
        
        # Commit the transaction
        conn.commit()
        print("The dataframe is inserted")
        
    except (Exception, psycopg2.DatabaseError) as error: 
        # Handle any errors
        print(f"Error: {error}")
        conn.rollback()
        
    finally:
        create_cursor.close()
        cursor.close()

In [64]:
import psycopg2
import pandas as pd

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)

# Define the SQL query
query = "SELECT * FROM silver_airline_quality_reviews;"

# Load the query result into a DataFrame
df_raw = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

/var/folders/dq/3b3sdc9971z_zvbxr0749v5h0000gn/T/ipykernel_41402/1848194661.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql_query(query, connection)


In [65]:
print(len(df_raw))
df_raw

24426


,Id,RowId,Airline Name,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Recommended,sentiment_label,sentiment_scores,Overall_Rating,is_lost_luggage_flag,is_lost_luggage_score,created_by,created_date,lastmodified_by,lastmodified_date
0,d5735d894b4092e822168ba53a488a01fea59947c7b592...,0,AB Aviation,"""pretty decent airline""",2019-11-11,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,yes,positive,0.918486,9.0,False,0.809639,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
1,7966382a7e6a7378845e6e7606deb05c3cd22ed48eb139...,1,AB Aviation,"""Not a good airline""",2019-06-25,True,Moroni to Anjouan. It is a very small airline...,NaN,E120,Solo Leisure,...,no,negative,0.569240,1.0,False,0.806561,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
2,b7272f236f9dd03ff3ddc7890a8c56ef10e4b453f87626...,2,AB Aviation,"""flight was fortunately short""",2019-06-25,True,Anjouan to Dzaoudzi. A very small airline an...,NaN,Embraer E120,Solo Leisure,...,no,neutral,0.542810,1.0,False,0.815163,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
3,2a1ca52e66cecb27b117aaa639137b03a903771bdc3581...,3,Adria Airways,"""I will never fly again with Adria""",2019-09-28,False,Please do a favor yourself and do not fly wi...,NaN,NaN,Solo Leisure,...,no,negative,0.903330,1.0,True,0.848293,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
4,0919fc2c8d770679071d9c4626486d411c386f1dcbcf1f...,4,Adria Airways,"""it ruined our last days of holidays""",2019-09-24,True,Do not book a flight with this airline! My fr...,NaN,NaN,Couple Leisure,...,no,negative,0.902472,1.0,False,0.811869,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24421,62e8ecb24d76161e74db6470c18e85426a461ae9a21441...,11,Discover Airlines,"“for the price, definitely not worth it”",2025-03-22,True,It all started when we checked in our luggage...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,no,neutral,0.764292,1.0,False,0.810353,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835
24422,911097fab091570462a7b30d91a3bf7b66abd4436b515d...,6,American Airlines,“may as well be a greyhound station”,2025-03-23,True,"My flight was originally for 6:15 am, this ...",NaN,NaN,Solo Leisure,...,no,negative,0.845816,1.0,False,0.814886,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835
24423,5b37c78f4bdf5ad19a471aed29f4977d2ade108724cbe1...,38,Sun Country Airlines,"""response was sorry for the delay""",2025-03-26,True,My flight was supposed to depart Tampa at 3...,NaN,NaN,Solo Leisure,...,no,negative,0.722160,1.0,False,0.790769,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835
24424,c1f1594af76bb5f31ca4aebeb9473c709113efaf5b239b...,5,American Airlines,"""delayed over 5 hours""",2025-03-27,True,I have flown five flights with American Air...,NaN,NaN,Business,...,no,negative,0.923790,1.0,False,0.790555,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835


In [68]:
def generate_hash(row):
    value = f"{row['Airline Name']}|{row['Review_Title']}|{row['Review Date']}"
    return hashlib.sha256(value.encode()).hexdigest()

In [70]:
df['Id'] = df.apply(generate_hash, axis=1)

In [72]:
print(len(df))
df

24426


,Id,RowId,Airline Name,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Recommended,sentiment_label,sentiment_scores,Overall_Rating,is_lost_luggage_flag,is_lost_luggage_score,created_by,created_date,lastmodified_by,lastmodified_date
0,b256efd586e017ce349a08be6743db65cfe0954bb00377...,0,AB Aviation,"""pretty decent airline""",2019-11-11,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,yes,positive,0.918486,9.0,False,0.809639,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
1,267ff04cb581496b15d0b76af0ed1b687aa629639fd66e...,1,AB Aviation,"""Not a good airline""",2019-06-25,True,Moroni to Anjouan. It is a very small airline...,NaN,E120,Solo Leisure,...,no,negative,0.569240,1.0,False,0.806561,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
2,c747d76f43b9deb57bd22aa906052ba0c8cd4269eedbbc...,2,AB Aviation,"""flight was fortunately short""",2019-06-25,True,Anjouan to Dzaoudzi. A very small airline an...,NaN,Embraer E120,Solo Leisure,...,no,neutral,0.542810,1.0,False,0.815163,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
3,d8e10cd13c248acff0f3e64b5edccd28bb0689adfd123b...,3,Adria Airways,"""I will never fly again with Adria""",2019-09-28,False,Please do a favor yourself and do not fly wi...,NaN,NaN,Solo Leisure,...,no,negative,0.903330,1.0,True,0.848293,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
4,a6b2b4520047b3e918c0c6a1ff3466ffd25267e021356f...,4,Adria Airways,"""it ruined our last days of holidays""",2019-09-24,True,Do not book a flight with this airline! My fr...,NaN,NaN,Couple Leisure,...,no,negative,0.902472,1.0,False,0.811869,From_AirlineQuality,2025-03-29 00:53:53.966539,From_AirlineQuality,2025-03-29 00:53:53.966539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24421,450185b67861624517d41550abd8b19e641e33f19c1b05...,11,Discover Airlines,"“for the price, definitely not worth it”",2025-03-22,True,It all started when we checked in our luggage...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,no,neutral,0.764292,1.0,False,0.810353,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835
24422,5499177406e54cbec8f0ac264eb4f7b48cb5befea8d981...,6,American Airlines,“may as well be a greyhound station”,2025-03-23,True,"My flight was originally for 6:15 am, this ...",NaN,NaN,Solo Leisure,...,no,negative,0.845816,1.0,False,0.814886,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835
24423,37e020bd45eb27e4c3f558946f153f67f4ba48f601d073...,38,Sun Country Airlines,"""response was sorry for the delay""",2025-03-26,True,My flight was supposed to depart Tampa at 3...,NaN,NaN,Solo Leisure,...,no,negative,0.722160,1.0,False,0.790769,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835
24424,1a0935fa87bcfd522c4661a50f2293d8094a9b4495d57e...,5,American Airlines,"""delayed over 5 hours""",2025-03-27,True,I have flown five flights with American Air...,NaN,NaN,Business,...,no,negative,0.923790,1.0,False,0.790555,From_AirlineQuality,2025-03-29 01:40:14.147835,From_AirlineQuality,2025-03-29 01:40:14.147835


In [74]:
conf ={
    'host':DB_HOST,
    'port':DB_PORT,
    'database':DB_NAME,
    'user':DB_USER,
    'password':DB_PASSWORD
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

df_raw.to_sql('temp_airline_quality_reviews_backupbeforefixId', engine, index=False, if_exists='replace')

426

In [102]:
conf ={
    'host':DB_HOST,
    'port':DB_PORT,
    'database':DB_NAME,
    'user':DB_USER,
    'password':DB_PASSWORD
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

df.to_sql('bronze_airline_quality_reviews', engine, index=False, if_exists='replace')

426

In [104]:
# This checks if record already exists by Id before inserting
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        TRUNCATE TABLE silver_airline_quality_reviews;
        INSERT INTO silver_airline_quality_reviews 
        (
        	"Id",
            "RowId",
        	"Airline Name",
        	"Review_Title",
        	"Review Date",
        	"Verified",
        	"Review",
        	"Top Review Image Url",
        	"Aircraft",
        	"Type Of Traveller",
        	"Seat Type",
        	"Route",
        	"Date Flown",
        	"Seat Comfort",
        	"Cabin Staff Service",
        	"Food & Beverages",
        	"Ground Service",
        	"Inflight Entertainment",
        	"Wifi & Connectivity",
        	"Value For Money",
        	"Recommended",
        	sentiment_label,
            sentiment_scores,
        	"Overall_Rating",
            is_lost_luggage_flag,
            is_lost_luggage_score,
            created_by,
            created_date,
            lastmodified_by,
            lastmodified_date           
        )
        SELECT
        	CAST("Id" AS varchar(64)),
            CAST("RowId" AS INT8),
        	"Airline Name",
        	"Review_Title",
        	CAST("Review Date" AS TIMESTAMP),
        	CAST("Verified" AS BOOL),
        	"Review",
        	"Top Review Image Url",
        	"Aircraft",
        	"Type Of Traveller",
        	"Seat Type",
        	"Route",
        	CAST("Date Flown" AS TIMESTAMP),
        	CAST("Seat Comfort" AS FLOAT8),
        	CAST("Cabin Staff Service" AS FLOAT8),
        	CAST("Food & Beverages" AS FLOAT8),
        	CAST("Ground Service" AS FLOAT8),
        	CAST("Inflight Entertainment" AS FLOAT8),
        	CAST("Wifi & Connectivity" AS FLOAT8),
        	CAST("Value For Money" AS FLOAT8),
        	"Recommended",
        	sentiment_label,
            CAST(sentiment_scores AS FLOAT8),
            CAST("Overall_Rating" AS FLOAT8),
            CAST(is_lost_luggage_flag AS BOOL),
            CAST(is_lost_luggage_score AS FLOAT8), 
            'From_AirlineQuality' AS created_by,
            NOW() AS created_date,
            'From_AirlineQuality' AS lastmodified_by,
            NOW() AS lastmodified_date
        FROM bronze_airline_quality_reviews AS S
    """)
    rowcount = cursor.rowcount

    cursor.execute("""
                DROP TABLE IF EXISTS bronze_airline_quality_reviews;
                """)
    
    connection.commit()
    print(f'{rowcount} records inserted to "silver_airline_quality_reviews" successfully')


except (Exception, psycopg2.Error) as error:
    print("Error during insert operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

24426 records inserted to "silver_airline_quality_reviews" successfully


In [100]:
import psycopg2
import pandas as pd

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)

# Define the SQL query
query = "SELECT * FROM silver_airline_quality_reviews;"

# Load the query result into a DataFrame
df_after = pd.read_sql_query(query, connection)

# Close the connection
conn.close()

/var/folders/dq/3b3sdc9971z_zvbxr0749v5h0000gn/T/ipykernel_41402/1216763083.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_after = pd.read_sql_query(query, connection)


In [106]:
print(len(df_after))
df_after

24426


,Id,RowId,Airline Name,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Recommended,sentiment_label,sentiment_scores,Overall_Rating,is_lost_luggage_flag,is_lost_luggage_score,created_by,created_date,lastmodified_by,lastmodified_date
0,b256efd586e017ce349a08be6743db65cfe0954bb00377...,0,AB Aviation,"""pretty decent airline""",2019-11-11,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,yes,positive,0.918486,9.0,False,0.809639,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
1,267ff04cb581496b15d0b76af0ed1b687aa629639fd66e...,1,AB Aviation,"""Not a good airline""",2019-06-25,True,Moroni to Anjouan. It is a very small airline...,NaN,E120,Solo Leisure,...,no,negative,0.569240,1.0,False,0.806561,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
2,c747d76f43b9deb57bd22aa906052ba0c8cd4269eedbbc...,2,AB Aviation,"""flight was fortunately short""",2019-06-25,True,Anjouan to Dzaoudzi. A very small airline an...,NaN,Embraer E120,Solo Leisure,...,no,neutral,0.542810,1.0,False,0.815163,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
3,d8e10cd13c248acff0f3e64b5edccd28bb0689adfd123b...,3,Adria Airways,"""I will never fly again with Adria""",2019-09-28,False,Please do a favor yourself and do not fly wi...,NaN,NaN,Solo Leisure,...,no,negative,0.903330,1.0,True,0.848293,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
4,a6b2b4520047b3e918c0c6a1ff3466ffd25267e021356f...,4,Adria Airways,"""it ruined our last days of holidays""",2019-09-24,True,Do not book a flight with this airline! My fr...,NaN,NaN,Couple Leisure,...,no,negative,0.902472,1.0,False,0.811869,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24421,450185b67861624517d41550abd8b19e641e33f19c1b05...,11,Discover Airlines,"“for the price, definitely not worth it”",2025-03-22,True,It all started when we checked in our luggage...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,no,neutral,0.764292,1.0,False,0.810353,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
24422,5499177406e54cbec8f0ac264eb4f7b48cb5befea8d981...,6,American Airlines,“may as well be a greyhound station”,2025-03-23,True,"My flight was originally for 6:15 am, this ...",NaN,NaN,Solo Leisure,...,no,negative,0.845816,1.0,False,0.814886,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
24423,37e020bd45eb27e4c3f558946f153f67f4ba48f601d073...,38,Sun Country Airlines,"""response was sorry for the delay""",2025-03-26,True,My flight was supposed to depart Tampa at 3...,NaN,NaN,Solo Leisure,...,no,negative,0.722160,1.0,False,0.790769,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
24424,1a0935fa87bcfd522c4661a50f2293d8094a9b4495d57e...,5,American Airlines,"""delayed over 5 hours""",2025-03-27,True,I have flown five flights with American Air...,NaN,NaN,Business,...,no,negative,0.923790,1.0,False,0.790555,From_AirlineQuality,2025-04-02 01:18:15.845890,From_AirlineQuality,2025-04-02 01:18:15.845890
